In [ ]:
import pandas as pd
dff = pd.read_csv('flavor_db.csv')[['entity id','alias','synonyms','scientific name','category','molecules']].set_index('entity id')

dff.head()

In [ ]:
dfm = pd.read_csv('molecules.csv')[['pubchem id','common name','flavor profile']].set_index('pubchem id').iloc[:-3]

dfm.head()

### food.com data

In [ ]:
import pandas as pd
dfr1 = pd.read_csv('food_dot_com/RAW_recipes.csv').rename(columns={'id':'recipe_id'})
dfr1=dfr1.set_index('recipe_id')

dfr1.head()

In [ ]:
import numpy as np
def g(df,col):
    df[col] = df[col].apply(eval).apply(lambda x: list(x))
    return df

temp = g(dfm.copy(),'flavor profile')
df_flavs=temp['flavor profile'].explode().replace('',np.nan).dropna()

df_flavs =df_flavs.reset_index().set_index(['pubchem id','flavor profile'])

df_flavs.head()

In [ ]:
dft = pd.Series(dfr1['ingredients'].apply(eval).explode().unique())

dft2 = dff.reset_index()

dft2['alias']=dft2['alias'].str.lower()

dft2=dft2.rename(columns = {'alias':'ingredients'})
df_filt=dft2[dft2['ingredients'].isin(dft)].copy()

df_filt['molecules'] = df_filt['molecules'].apply(eval)


def g(df):
    df["molecules"] = df["molecules"].apply(lambda x: list(x))
    return df

df = g(df_filt.copy())
# Step 1: Explode the 'molecules' column
df_exploded = df.set_index('ingredients').explode('molecules')

# Step 2: Create a binary matrix
# Convert the exploded DataFrame into a binary matrix where each molecule has its own column
# and the presence of the molecule is indicated by 1
df_matrix = pd.get_dummies(df_exploded['molecules']).groupby(df_exploded.index).sum()

df_matrix.head()

In [ ]:
# Ensure 'ingredients' are evaluated to lists and filter out empty lists before exploding
dfr1['ingredients'] = dfr1['ingredients'].apply(eval)
dfr1 = dfr1[dfr1['ingredients'].map(bool)]  # Filter out rows where 'ingredients' is an empty list


In [ ]:
temp_df=dfr1['ingredients'].explode().reset_index().set_index(['recipe_id', 'ingredients'])

In [ ]:
# Example of processing in batches
batch_size = 10000  # Adjust based on your system's memory capacity
n_batches = (len(dfr1) + batch_size - 1) // batch_size  # Calculate the number of batches needed

result_dfs = []  # List to hold the result of each batch

for i in range(n_batches):
    batch_df = dfr1.iloc[i*batch_size:(i+1)*batch_size]  # Select a batch
    batch_df = batch_df.explode('ingredients').reset_index().dropna().set_index(['recipe_id', 'ingredients'])
    batch_result = batch_df.join(df_matrix, on='ingredients', how='left').fillna(0).groupby('recipe_id').sum()
    result_dfs.append(batch_result)

# Combine batch results
df_agg_mol = pd.concat(result_dfs)

In [ ]:
import pandas as pd
dfr2 = pd.read_csv('food_dot_com/RAW_interactions.csv').set_index(['recipe_id','user_id'])

dfr2['review']=dfr2['review'].astype(str)

flavor_words = {
    'acidic': ['sharp', 'tart', 'sour'],
    'bitter': ['tart', 'sharp', 'harsh'],
    'sweet': ['sugary'],
    'umami': ['full-flavored', 'savory'],
    'spicy': ['pungent', 'hot'],
}
# Tokenize the reviews and count flavor words
for flavor, words in flavor_words.items():
    dfr2[flavor] = dfr2['review'].str.lower().str.count('|'.join(words))

In [ ]:
dfr2.head()

In [ ]:
df_agg = dfr2.groupby('recipe_id').agg({'acidic':'sum',
                                         'bitter':'sum',
                                         'sweet':'sum',
                                         'umami':'sum',
                                         'spicy':'sum'})

df_agg=df_agg.rename(columns = {'acidic':'acidic_rev',
                                         'bitter':'bitter_rev',
                                         'sweet':'sweet_rev',
                                         'umami':'umami_rev',
                                         'spicy':'spicy_rev'})

df_final = df_agg.join(df_agg_mol,how='left')

df_final.head()

In [ ]:
df_final.to_pickle('full_dataset.pkl')

-----------------------